In [1]:
%cd ../..
# %pip install .
import time
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50, 
#         with viewAngle 135 deg
# ped = Pedestrian on roadsOrIntersections,
#         with regionContainedIn roadRegion,
#         facing Range(-180, 180) deg

# require abs(relative heading of ped from ego) > 70 deg

c:\Users\youse\Desktop\Research\Apperception\apperception


In [2]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'pedestrian'
) as pedestrians

select *
from pedestrians as t1
join Cameras as cam on t1.cameraId = Cameras.id
where
  t1.heading < 180 and t1.heading > -180 and
  (contained(t1.traj, road_segment("roads"), cam.timestamp) or
   contained(t1.traj, road_segment("Intersections"), cam.timestamp)) and
  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -70 OR
  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > 70) AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND
  viewAngle(t1.traj, cam.egoHeading, cam.ego_translation, cam.timestamp) < 135
"""

'\nwith (\n  select *\n  from item_traj\n  where item_traj.object_type = \'pedestrian\'\n) as pedestrians\n\nselect *\nfrom pedestrians as t1\njoin Cameras as cam on t1.cameraId = Cameras.id\nwhere\n  t1.heading < 180 and t1.heading > -180 and\n  (contained(t1.traj, road_segment("roads"), cam.timestamp) or\n   contained(t1.traj, road_segment("Intersections"), cam.timestamp)) and\n  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -70 OR\n  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > 70) AND\n  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND\n  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND\n  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND\n  viewAngle(t1.traj, cam.egoHeading, cam.ego_translation, cam.timestamp) < 135\n'

In [6]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
world = world.filter(lambda obj: obj.object_type == 'human.pedestrian.adult' or
                                 obj.object_type == 'human.pedestrian.wheelchair' or
                                 obj.object_type == 'human.pedestrian.personal_mobility' or
                                 obj.object_type == 'human.pedestrian.construction_worker' or
                                 obj.object_type == 'human.pedestrian.police_officer' or
                                 obj.object_type == 'human.pedestrian.child' or
                                 obj.object_type == 'human.pedestrian.stroller')
def pred(obj1, cam):
    return (
        obj1.heading < 180 and obj1.heading > -180 and
        (F.contained(obj1.traj, F.road_segment("road"), cam.timestamp) or
            F.contained(obj1.traj, F.road_segment("intersection"))) and
        (F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or
            F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and
        F.viewAngle(obj1, cam.ego, cam.timestamp) < 135
    )
world = world.filter("lambda obj1, cam: F.facing_relative(obj1, 0, cam.timestamp) < 180 and " + 
                     "F.facing_relative(obj1, 0, cam.timestamp) > -180 and " +
        "(F.contained(obj1.traj, F.road_segment('road'), cam.timestamp) or " + 
            "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp)) and " +
        "(F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or " + 
            "F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and " + 
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and " +
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
        "F.view_angle(obj1, cam.ego, cam.timestamp) < 135")

start = time.time()

key = world.get_traj_key()

end = time.time()
print(format(end-start))

execute: filter
execute: filter
execute: get_traj_key
get_traj_key 
        SELECT itemId FROM (
        SELECT DISTINCT table_0.*
        FROM (
        SELECT DISTINCT table_0.*
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE ((table_0.objectType='human.pedestrian.adult') OR (table_0.objectType='human.pedestrian.wheelchair') OR (table_0.objectType='human.pedestrian.personal_mobility') OR (table_0.objectType='human.pedestrian.construction_worker') OR (table_0.objectType='human.pedestrian.police_officer') OR (table_0.objectType='human.pedestrian.child') OR (table_0.objectType='human.pedestrian.stroller'))
        ) as table_0
        
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((facingRelative(table_0.itemHeadings, 0, Cameras.timestamp)<180) AND (facingRelative(table_0.itemHeadings, 0, Cameras.timestamp)>(-180)) AND (contained(table_0.trajCentroids, roadSegment('road'), Cameras.timestamp) OR contained(table_

In [8]:
keys = world.get_traj_key()
trajs = world.get_traj()
scene_names = world.get_traj_attr("cameraId")
print(keys)
for i in range(len(keys)):
    key = keys[i][0]
    traj = trajs[i][0]
    scene_name = scene_names[i][0]
    print(world.get_trajectory_images(scene_name, traj))

execute: filter
execute: filter
execute: get_traj_key
get_traj_key 
        SELECT itemId FROM (
        SELECT DISTINCT table_0.*
        FROM (
        SELECT DISTINCT table_0.*
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE ((table_0.objectType='human.pedestrian.adult') OR (table_0.objectType='human.pedestrian.wheelchair') OR (table_0.objectType='human.pedestrian.personal_mobility') OR (table_0.objectType='human.pedestrian.construction_worker') OR (table_0.objectType='human.pedestrian.police_officer') OR (table_0.objectType='human.pedestrian.child') OR (table_0.objectType='human.pedestrian.stroller'))
        ) as table_0
        
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((facingRelative(table_0.itemHeadings, 0, Cameras.timestamp)<180) AND (facingRelative(table_0.itemHeadings, 0, Cameras.timestamp)>(-180)) AND (contained(table_0.trajCentroids, roadSegment('road'), Cameras.timestamp) OR contained(table_